In [ ]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# import the libraries

import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import neighbors
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

***Clean the data***

In [ ]:
# read the data and normilize it
df=pd.read_csv('main_50k_pixels.csv',encoding='latin-1', delimiter=";", low_memory=False)

FileNotFoundError: ignored

In [ ]:
df.shape

In [ ]:
del df['luc']

In [ ]:
df_norm=df.convert_objects(convert_numeric=True).apply(lambda x: (x-x.min())/np.ptp(x)).dropna(how='any', axis=0)

In [ ]:
df_norm.shape

###df.columns.tolist()

In [ ]:
# create 3 subsets for different years
df1=df_norm[df_norm.columns.drop(list(df_norm.filter(regex='id2|Unnamed: 0|pobtot1973|vivocupada1973|2000|dfedge.86|dfedge.97|for.86|for.97|pro.aft80per|popgrowth')))]
df2=df_norm[df_norm.columns.drop(list(df_norm.filter(regex='id2|Unnamed: 0|pobtot1973|vivocupada1973|2000|dfedge.60|dfedge.97|for.60|for.97|pro.aft80per|popgrowth')))]
df3=df_norm[df_norm.columns.drop(list(df_norm.filter(regex='id2|Unnamed: 0|poptot2000|pobtot1973|empty.2000|pop.2000|73|1960|dfedge.60|dfedge.86|for.60|for.86|pro.bef80per|pro.before2000per|popgrowth')))]

In [ ]:
print(df1.shape)
print(df2.shape)
print(df3.shape)

In [ ]:
F1 = df1['for.60']
F2 = df2['for.86']
F3 = df3['for.97']

In [ ]:
(df1['for.60'] != 0).sum()

In [ ]:
(df1['for.60'] != 1).sum()

In [ ]:
(df2['for.86'] != 0).sum()

In [ ]:
(df2['for.86'] != 1).sum()

In [ ]:
(df3['for.97'] != 1).sum()

In [ ]:
(df3['for.97'] != 0).sum()

In [ ]:
# Rearrange the data to have predictor as a first column
df1.drop(labels=['for.60'], axis=1,inplace = True)
df1.insert(0, 'for.60', F1)
df2.drop(labels=['for.86'], axis=1,inplace = True)
df2.insert(0, 'for.86', F2)
df3.drop(labels=['for.97'], axis=1,inplace = True)
df3.insert(0, 'for.97', F3)

In [ ]:
print(df1.shape)
print(df2.shape)
print(df3.shape)

In [ ]:
df.head(15)


***Classification***

In [ ]:
# Choose the classification methods

X1 = df1.values
X2 = df2.values
X3 = df3.values

gnb = GaussianNB()
qda = QDA()
lda = LDA()
knn = neighbors.KNeighborsClassifier(n_neighbors = 3)
dt = tree.DecisionTreeClassifier(criterion= 'gini', max_depth= None)
rf = RF(criterion= 'gini')

names = ['GaussianNB', 'QDA', 'LDA', 'kNN','Decision Tree','Random Forest']
classifiers = [gnb, qda, lda, knn, dt, rf]

In [ ]:
print(X1.shape)

In [ ]:
# Classification within the same year
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1[:,1:54], X1[:,0], test_size=0.33,random_state=1)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2[:,1:54], X2[:,0], test_size=0.33,random_state=1)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3[:,1:54], X3[:,0], test_size=0.33,random_state=1)

for name, classifier in zip(names, classifiers):
    classifier.fit(X_train1, y_train1)
    pred1 = classifier.predict(X_test1)
    print('{} -- Number of mislabeled points out of total {} points: {}'.format(name, len(y_test1), (pred1 != y_test1).sum()))
print()
for name, classifier in zip(names, classifiers):
    classifier.fit(X_train2, y_train2)
    pred2 = classifier.predict(X_test2)
    print('{} -- Number of mislabeled points out of total {} points: {}'.format(name, len(y_test2), (pred2 != y_test2).sum()))
print()
for name, classifier in zip(names, classifiers):
    classifier.fit(X_train3, y_train3)
    pred3 = classifier.predict(X_test3)
    print('{} -- Number of mislabeled points out of total {} points: {}'.format(name, len(y_test3), (pred3 != y_test3).sum()))

In [ ]:
# Confusion matrix and prediction rate
from sklearn.metrics import confusion_matrix
print(confusion_matrix (y_test1,pred1))
print ('Predicted\nError Rate = {:0.2f}%'.format((pred1 != y_test1).sum()*100/len(y_test1)))
print(confusion_matrix (y_test2,pred2))
print ('Predicted\nError Rate = {:0.2f}%'.format((pred2 != y_test2).sum()*100/len(y_test2)))
print(confusion_matrix (y_test3,pred3))
print ('Predicted\nError Rate = {:0.2f}%'.format((pred3 != y_test3).sum()*100/len(y_test3)))

***Prediction***

*Prediction of 1980 using 1960*

In [ ]:
# Prediction of 1980 using 1960
X_train, X_test, y_train, y_test = train_test_split(X1[:,1:54], X2[:,0], test_size=0.33,random_state=1)



for name, classifier in zip(names, classifiers):
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    print('{} -- Number of mislabeled points out of total {} points: {}'.format(name, len(y_test), (pred != y_test).sum()))
print()


In [ ]:
print ('Predicted\nError Rate = {:0.2f}%'.format((pred != y_test).sum()*100/len(y_test)))
print(confusion_matrix (y_test,pred))

In [ ]:
# Decision tree best predictors
feat_importances1 = pd.Series(dt.feature_importances_, index=df1.columns[1:54])
feat_importances1.nlargest(20).plot(kind='barh')

In [ ]:
# Random forest best predictors
feat_importances = pd.Series(rf.feature_importances_, index=df1.columns[1:54])
feat_importances.nlargest(20).plot(kind='barh')

In [ ]:
# Random forest best predictors
feat_importances = pd.Series(rf.feature_importances_, index=df1.columns[1:54])
feat_importances.nlargest(7).plot(kind='barh')

In [ ]:
# Best 7 predictors of RF
feat_importances.nlargest(7)

***Prediction of 1980 with the best 7 predictors ***


In [ ]:
# New data set that only contains best predictive variables from 1960
new_df1 = df1[['drd.00','drd','dfedge.60','for60per','dmcity','seg.area','slope']].copy()
new_df1[:4]
new_df1.shape

In [ ]:
X11 = new_df1.values
# Prediction of 1980 using 1960
XX_train, XX_test, yy_train, yy_test = train_test_split(X11[:,1:9], X2[:,0], test_size=0.33,random_state=1)



for name, classifier in zip(names, classifiers):
    classifier.fit(XX_train, yy_train)
    pred4 = classifier.predict(XX_test)
    print('{} -- Number of mislabeled points out of total {} points: {}'.format(name, len(yy_test), (pred4 != yy_test).sum()))
print()

In [ ]:
print ('Predicted\nError Rate = {:0.2f}%'.format((pred4 != yy_test).sum()*100/len(yy_test)))

In [ ]:
# Confusion matrix

confusion_matrix (yy_test,pred4)


*Prediction of 2000 using the previous model*

In [ ]:
# Prediction of 2000 using 1980
X60_train = X1[:,1:54]
Y80_train = X2[:,0]
X80_test = X2[:,1:54]
Y2000_test = X3[:,0]

for name, classifier in zip(names, classifiers):
    classifier.fit(X60_train, Y80_train)
    pred2000 = classifier.predict(X80_test)
    print('{} -- Number of mislabeled points out of total {} points: {}'.format(name, len(Y2000_test), (pred2000 != Y2000_test).sum()))
print()

In [ ]:
confusion_matrix (Y2000_test,pred2000)

In [ ]:
print ('Predicted\nError Rate = {:0.2f}%'.format((pred2000 != Y2000_test).sum()*100/len(Y2000_test)))

*ROC Curve*

In [ ]:
import random
random.seed(123)
random.shuffle(X1)
random.shuffle(X11)

XX1 = X1[:int(len(X1.T))]
XX2 = X11[:int(len(X11.T))]
print(XX1.shape)
print(XX2.shape)

In [ ]:
#plot orig and new covariance matrices(estimate w/o norm)
figure(figsize=(10,5))
subplot(121);imshow(XX1.dot(XX1.T),interpolation = 'none',cmap = 'viridis');
plt.title('Covariance Matrix for 1960')

In [ ]:
figure(figsize=(10,5))
subplot(122);imshow(XX2.dot(XX2.T),interpolation = 'none',cmap = 'viridis');
plt.title('Covariance Matrix for best 7 Predictors')

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(pred2000, Y2000_test)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for 2000 forestation prediction')
plt.legend(loc="lower right")
plt.show()